In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import timeit

In [ ]:
#Function to get the list of pages from 1-10 in goodreads website
def get_list_pages_of_10(url_list_10):
    url_list = []
    for page_number in range(0,11):
        #remember that the end is ?page=
        url_page = url_list_10+str(page_number)
        url_list.append(url_page)
    return url_list

In [ ]:
list_of_10pages = get_list_pages_of_10('https://www.goodreads.com/list/show/1.Best_Books_Ever?page=')

In [ ]:
# function to get href 100 links
def get_links_for_100(url_one_page):
    url_page = requests.get(url_one_page)
    soup = BeautifulSoup(url_page.content, 'html.parser')
    href_link_list =[]
    for link in soup.find_all('a', class_='bookTitle') :
        if link.has_attr('href'):
            href_link_list.append("https://www.goodreads.com" + str(link.attrs['href']))
    return href_link_list

In [ ]:
list_of_1000_links = get_all_1000_links('https://www.goodreads.com/list/show/1.Best_Books_Ever?page=')

In [ ]:
def get_all_1000_links(url_list):
    url_10_pages = get_list_pages_of_10(url_list)
    all_links = []
    for link in url_10_pages:
        all_links.append(get_links_for_100(link))
    flat_list_of_1000 = []
    for elem in all_links:
        flat_list_of_1000.extend(elem)
    return flat_list_of_1000

In [ ]:
%timeit list_of_1000_links = get_all_1000_links('https://www.goodreads.com/list/show/1.Best_Books_Ever?page=')

In [ ]:
print(list_of_1000_links)

In [ ]:
print(len(list_of_100_links))
print(list_of_100_links[3])

In [21]:
# get title
def get_title(soup_detail):
    try:
        title = soup_detail.find("h1", {"id":"bookTitle"}).text.replace("\n","").replace("      ","")
    except:
        title = np.nan
    return title

# get author
def get_author(soup_detail):
    try:
        author = soup_detail.find("div", {"class":"authorName__container"}).text.replace("\n","")
    except:
        author = np.nan
    return author





# get ratings
def get_rating(soup_detail):
    try:
        rating = soup_detail.find("meta", {"itemprop":"ratingCount"}).text.replace("\n","").split()
        rating = rating[0]        
    except:
        rating = np.nan      
    return rating


# get avgratings
book_avgratings = []
def get_avgrating(soup_detail):
    try:
        avgrating = soup_detail.find("span", {"itemprop":"ratingValue"}).text.replace("\n","").replace(' ','')
    except:
        avgrating = np.nan
    return avgrating

# get pages
book_pages = []
def get_page(soup_detail):
    try:
        page = soup_detail.find("span", {"itemprop":"numberOfPages"}).text.split()
        page = page[0]
    except:
        page = np.nan
    return page


# get years
book_publish_year = []
def get_year(soup_detail):
    try:
        publish_year = soup_detail.find("nobr", {"class":"greyText"}).text.replace("\n","").replace(")","").split()
        publish_year = publish_year[-1]
    except:
        publish_year = np.nan
    return publish_year

# get award
def get_award(soup_detail):
    try:
        award = soup_detail.find("div", {"itemprop":"awards"}).text.replace("\n","")
    except:
        award = np.nan
    return award

# get genres
def get_genres(soup_detail):
    try:
        genres = soup_detail.find_all(class_="actionLinkLite bookPageGenreLink")
        genres = genres[:3]
        genres = [genre.get_text() for genre in genres]
    except:
        genres = np.nan
    return genres

#get the book reviews
def get_review(soup_detail):
    try:
        review = soup_detail.find("meta", {"itemprop":"reviewCount"}).text.replace("\n","").split()
        review = review[0]
    except:
        review = np.nan
    return review

# get series
def get_series(soup_detail):
    try:
        series = soup_detail.find(id="bookSeries").text.strip()
        if len(series) == 0:
            series = False
        else: 
            series = True
    except:
        series = np.nan
    return series

# get places
def get_places(soup_detail):
    try:
        places = soup_detail.find("div", {'id':"bookDataBox"}).find('span',class_="darkGreyText").text.replace("(","").replace(")","").strip()
    except:
        places = np.nan
    return places

In [27]:
def create_df(url_list):
    title = []
    author = []
    num_reviews = []
    num_ratings = []
    avg_rating = []
    num_pages = []
    original_publish_year = []
    series = []
    genres = []
    awards = []
    places = []
    
    for link in url_list:
        one_book= requests.get(link)
        soup_detail = BeautifulSoup(one_book.content, 'html.parser')
    

        title.append(get_title(soup_detail))
        author.append(get_author(soup_detail))
        num_reviews.append(get_review(soup_detail))
        num_ratings.append(get_rating(soup_detail))
        avg_rating.append(get_avgrating(soup_detail))
        num_pages.append(get_page(soup_detail))
        original_publish_year.append(get_year(soup_detail))
        series.append(get_series(soup_detail))
        genres.append(get_genres(soup_detail))
        awards.append(get_award(soup_detail))
        places.append(get_places(soup_detail))
        
    data = {'url' : url_list,
            'title':title,
            'author':author,
            'num_reviews':num_reviews,
            'num_ratings': num_ratings,
            'avg_rating':avg_rating,
            'num_pages':num_pages,
            'original_publish_year':original_publish_year,
            'series':series,
            'genres':genres,
            'awards':awards,
            'places':places
           }
    df = pd.DataFrame(data)
    df.to_csv("df_new_1100.csv", index=True)

In [28]:
###### run webscrapper for 10 books!
def get_links_for_10(url_one_page):
    url_page = requests.get(url_one_page)
    soup = BeautifulSoup(url_page.content, 'html.parser')
    href_link_list =[]
    n = 0
    while n < 10:
        for link in soup.find_all('a', class_='bookTitle') :
            if link.has_attr('href'):
                href_link_list.append("https://www.goodreads.com" + str(link.attrs['href']))
                n += 1
    return href_link_list

In [29]:
#list_of_100_links = get_links_for_100('https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1')
list_of_10_links = list_of_100_links[:10] 
print(len(list_of_10_links)

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_12576/1952549587.py, line 3)